In [1]:
%matplotlib inline
from effective_quadratures.PolyParams import PolynomialParam
from effective_quadratures.PolyParentFile import PolyParent
from effective_quadratures.IndexSets import IndexSet
import effective_quadratures.Integrals as integrals
import effective_quadratures.Utils as utils
import matplotlib.pyplot as plt
import numpy as np

<h1> Numerical Integration


Numerical integration forms a pivotal component of modern computational engineering. In this notebook, we will test out a few of the integration routines in Effective-Quadratures. Consider the following integral
$$\int_{-2}^{1}\int_{-3}^{2}\int_{-1}^{2}\int_{0.5}^{0.5}\left(cos\left(x_{1}\right)+x_{2}^{2}+x_{3}x_{4}\right)dx_{1}dx_{2}dx_{3}dx_{4}.
$$

We will approximate the integral using three different techniques:
    1. Tensor grid
    2. Sparse grid
    3. Effective quadrature subsampling (least squares)
    4. Monte Carlo sampling
To begin, lets define the function and its bounds

In [2]:
def integral(x):
    return np.cos(x[0]) + x[1]**2 + x[2]*x[3]

<h2> 1. Tensor grid

In [3]:
no_of_pts = 4
dimensions = 4
parameter_x1 = PolynomialParam("Uniform", 0.5, 0.5, [], [], [], no_of_pts)
parameter_x2 = PolynomialParam("Uniform", -1.0, 2.0, [], [], [], no_of_pts)
parameter_x3 = PolynomialParam("Uniform", -3.0, 2.0, [], [], [], no_of_pts)
parameter_x4 = PolynomialParam("Uniform", -2.0, 1.0, [], [], [], no_of_pts)
allParameters = [parameter_x1, parameter_x2, parameter_x3, parameter_x4]

Now lets generate a tensor grid index set and hand that over to the integrals function:

In [4]:
tensorgridObject = IndexSet("tensor grid", [no_of_pts, no_of_pts, no_of_pts, no_of_pts])
tg_pts, tg_wts = integrals.tensorGrid(allParameters, tensorgridObject)

Computing the integral yields:

In [5]:
tensor_approximation = np.mat(tg_wts) * utils.evalfunction(tg_pts, integral)
tensor_grid_cardinality = IndexSet.getCardinality(tensorgridObject)
print 'TENSOR GRID APPROXIMATION & POINTS USED'
print tensor_approximation, tensor_grid_cardinality

TENSOR GRID APPROXIMATION & POINTS USED
[[ 34.04132099]] 625


<h2> 2. Sparse grid

We will use an exponential growth rule with level 3 to approximate the integral:

In [6]:
sparsegridObject = IndexSet("sparse grid", [], 4, "exponential", dimensions)
sg_pts, sg_wts = integrals.sparseGrid(allParameters, sparsegridObject)
sparse_int = np.mat(sg_wts) * utils.evalfunction(sg_pts, integral)
sparse_grid_cardinality = IndexSet.getCardinality(sparsegridObject)

In [7]:
print 'SPARSE GRID APPROXIMATION & POINTS USED'
print sparse_int, sparse_grid_cardinality

SPARSE GRID APPROXIMATION & POINTS USED
[[ 34.04132099]] 69
